# Environment for specific POMDP model

In [2]:
from StormExecutor import StormExecutor, _build_sensor_selection_const, _build_world_definition_const, \
    _define_program_constants, _configure_buildfull_options
from builders.POMDPAdapter import POMDPAdapter
from builders.ssp import LineTPMC
import stormpy
import stormpy.pomdp
import stormpy.pars

tpmc = LineTPMC(budget=30, goal=30, length=61)
pomdp = POMDPAdapter(tpmc)
exec = StormExecutor(verbose=True, puzzle_type=pomdp.puzzle_type)


obs_function = [1] + [1]*29 + [-1] + [0]*30
constants = {
    **_build_sensor_selection_const(obs_function, 50),
    **_build_world_definition_const(pomdp),
}

# Instantiate all constant parameters in the PRISM model
static_program = _define_program_constants(exec.static_program, constants)

# Build the parsed POMDP model with full labels and rewards (make it canonic)
build_options = _configure_buildfull_options()
model = stormpy.build_sparse_exact_model_with_options(static_program, build_options)
model = stormpy.build_sparse_model_with_options(static_program, build_options)
model = stormpy.pomdp.make_canonic(model)

### Parameter Synthesis using Region Verification (weak)

In [2]:
# Convert POMDP to parametric model using unknown FSC
parametric_model = stormpy.pomdp.apply_unknown_fsc(
    model,
    stormpy.pomdp.PomdpFscApplicationMode.simple_linear
)

In [ ]:
# Option 1: Parameter synthesis (requires bounded reward property <= tau)
env = stormpy.Environment()
checker = stormpy.pars.create_region_checker(env, parametric_model, exec.property[0].raw_formula)
parameters = parametric_model.collect_probability_parameters()
print(parameters)

valuations = { parameter: (0, 1) for parameter in parameters }
region = stormpy.pars.ParameterRegion(valuations)
synthesis_result = checker.check_region(env, region)

# Parametric MC with Gradient Descent for Memory-less POMDP Controllers

To integrate a parametric Markov chain in a Gradient Descent algorithm for computing memory-less POMDP controllers, one needs to combine `stormpy`'s parametric analysis capabilities with optimization logic.

## Core Approach
The key is to use the parametric model from `apply_unknown_fsc` as a differentiable function that maps controller parameters to verification results, then apply gradient descent to optimize these parameters. Using the `simple_linear` FSC application mode, we enforce a POMDP to pMC conversion that assumes no memory states (i.e. memory-less controllers).

Implementation Steps
1. Create Parametric Model with Unknown FSC

In [3]:
# Convert POMDP to parametric model using unknown FSC
parametric_model = stormpy.pomdp.apply_unknown_fsc(
    model,
    stormpy.pomdp.PomdpFscApplicationMode.simple_linear
)

2. Extract Parameters and Derivatives

In [4]:
# Collect model parameters
parameters = parametric_model.collect_probability_parameters()

print(parameters)
print(parametric_model)

# Function to compute gradients
def compute_gradients(parametric_model, parameters):
    gradients = {}
    for param in parameters:
        # Gather derivatives for each parameter
        derivatives = stormpy.pars.gather_derivatives(parametric_model, param)
        gradients[param] = derivatives
    return gradients

print(compute_gradients(parametric_model, parameters))

{<Variable p14_0 [id = 3]>, <Variable p32_0 [id = 4]>, <Variable p22_0 [id = 5]>, <Variable p20_0 [id = 6]>, <Variable p27_0 [id = 7]>, <Variable p28_0 [id = 8]>, <Variable p18_0 [id = 9]>, <Variable p2_0 [id = 10]>, <Variable p29_0 [id = 11]>, <Variable p5_0 [id = 12]>, <Variable p12_0 [id = 13]>, <Variable p8_0 [id = 14]>, <Variable p4_0 [id = 15]>, <Variable p19_0 [id = 16]>, <Variable p21_0 [id = 17]>, <Variable p10_0 [id = 18]>, <Variable p23_0 [id = 19]>, <Variable p6_0 [id = 20]>, <Variable p25_0 [id = 21]>, <Variable p24_0 [id = 22]>, <Variable p11_0 [id = 23]>, <Variable p3_0 [id = 24]>, <Variable p26_0 [id = 25]>, <Variable p7_0 [id = 26]>, <Variable p17_0 [id = 27]>, <Variable p13_0 [id = 28]>, <Variable p30_0 [id = 29]>, <Variable p9_0 [id = 30]>, <Variable p16_0 [id = 31]>, <Variable p31_0 [id = 32]>, <Variable p0_0 [id = 33]>, <Variable p1_0 [id = 34]>, <Variable p1_1 [id = 35]>}
-------------------------------------------------------------- 
Model type: 	DTMC (sparse)
St

In [6]:
import stormpy.pycarl.cln

# Function to evaluate gradients using current values for parameters
def evaluate_gradient(gradient_formulas: set, values: dict):
    if len(gradient_formulas) == 0:
        return 0
    else:
        gradient_polynomial = gradient_formulas.pop()
        grad_val = gradient_polynomial.evaluate(values)
        return float(grad_val)

3. Gradient Descent Optimization Loop (Baseline)

**Update Rule:**
```
θ_{t+1} = θ_t - η ∇f(θ_t)
```

Where:
- $\theta_t$: parameters at iteration $t$
- $\eta$: learning rate (step size)
- $\nabla f(\theta_t)$: gradient of objective function at $\theta_t$


In [7]:
def gradient_descent_optimization(parametric_model, formula, parameters,
                                learning_rate=0.01, max_iterations=100):
    # Initialize parameters
    current_values = {p: stormpy.RationalRF(0.5) for p in parameters}
    instantiator = stormpy.pars.ModelInstantiator(parametric_model)

    for iteration in range(max_iterations):
        # Instantiate model with current parameters
        concrete_model = instantiator.instantiate(current_values)

        # Model check to get objective value
        result = stormpy.model_checking(concrete_model, formula)
        objective = float(result.at(concrete_model.initial_states[0]))

        # Compute gradients (symbolic differentiation)
        gradients = compute_gradients(parametric_model, parameters)

        # Update parameters using gradient descent
        for param in parameters:
            # print(f"Parameter: {param}, Gradient: {gradients[param]}, Value: {float(current_values[param])}")
            # Convert symbolic gradient to numeric
            grad_value = evaluate_gradient(gradients[param], current_values)
            current_values[param] = stormpy.RationalRF(
                float(current_values[param]) - learning_rate * grad_value
            )

        print(f"Iteration {iteration}: Objective = {objective}")

    return current_values


4. Run Gradient Descent - extract memory-less controller

In [21]:
# Check CLN availability
from stormpy import pycarl
if not pycarl.has_cln():
    raise ImportError("CLN backend not available")

In [10]:
# Run gradient descent
optimal_parameters = gradient_descent_optimization(
    parametric_model, exec.property[0].raw_formula, parameters,
    learning_rate=0.05, max_iterations=50
)

# Get final controller
instantiator = stormpy.pars.ModelInstantiator(parametric_model)
optimal_model = instantiator.instantiate(optimal_parameters)
final_result = stormpy.model_checking(optimal_model, exec.property[0].raw_formula)

Iteration 0: Objective = 630.3333333333331
Iteration 1: Objective = 630.3333333333331
Iteration 2: Objective = 630.3333333333331
Iteration 3: Objective = 630.3333333333331
Iteration 4: Objective = 630.3333333333331
Iteration 5: Objective = 630.3333333333331
Iteration 6: Objective = 630.3333333333331
Iteration 7: Objective = 630.3333333333331
Iteration 8: Objective = 630.3333333333331
Iteration 9: Objective = 630.3333333333331
Iteration 10: Objective = 630.3333333333331
Iteration 11: Objective = 630.3333333333331
Iteration 12: Objective = 630.3333333333331
Iteration 13: Objective = 630.3333333333331
Iteration 14: Objective = 630.3333333333331
Iteration 15: Objective = 630.3333333333331
Iteration 16: Objective = 630.3333333333331
Iteration 17: Objective = 630.3333333333331
Iteration 18: Objective = 630.3333333333331
Iteration 19: Objective = 630.3333333333331
Iteration 20: Objective = 630.3333333333331
Iteration 21: Objective = 630.3333333333331
Iteration 22: Objective = 630.333333333333


### Theoretical Foundations of Standard Gradient Descent (Baseline)

**Update Rule:**
```
θ_{t+1} = θ_t - η ∇f(θ_t)
```

Where:
- $\theta_t$: parameters at iteration $t$
- $\eta$: learning rate (step size)
- $\nabla f(\theta_t)$: gradient of objective function at $\theta_t$

**Limitations:**
- **Slow convergence** in ravines (regions where surface curves more steeply in one dimension than another)
- **Oscillation** across narrow valleys
- **Fixed learning rate** may be too large (divergence) or too small (slow convergence)
- **Saddle points**: can get stuck where gradient is zero but not at optimum

---

### Key Components

>   `apply_unknown_fsc`: Creates parametric model with FSC parameters from POMDP

>   ModelInstantiator: Instantiates parametric models with specific values

>   `gather_derivatives`: Computes symbolic derivatives of transition probabilities

>   Parametric model checking: Returns symbolic results for gradient computation

### Memory-less Strategy Enforcement

The `SIMPLE_LINEAR` FSC application mode ensures memory-less strategies by:

- Using a linear parameterization without memory states
- Mapping observations directly to action probabilities
- Enforcing that the controller depends only on current observations

### Notes

- Stormpy provides the parametric model and derivative computation, but the gradient descent algorithm must be implemented in Python
- The `gather_derivatives` function returns symbolic derivatives that need to be evaluated numerically (**issue: not always computed for each parameter**)
- Learning rate and convergence criteria should be tuned based on the specific POMDP
- The approach works for reachability and probability properties; reward properties may need different handling

### Conclusion
At each step the derivatives extracted from stormpy are zero for most parameters, hence they seem to not affect the reward objective, which is in fact unrealistic. The problem might be too complex for symbolic derivatives and potentially not be captured by gradient-based methods. Alternatives: random search or evolutionary algorithms or policy gradient (REINFORCE with trajectory sampling).

# Advanced Gradient Descent Optimizers

Below are enhanced optimization algorithms with momentum and adaptive learning rates:

In [ ]:
import numpy as np

# Improved gradient evaluation that handles multiple derivatives
def evaluate_gradient_improved(gradient_formulas: set, values: dict):
    """
    Evaluate all gradient polynomials and aggregate them (sum or average).
    This handles cases where multiple derivatives exist for a parameter.
    """
    if len(gradient_formulas) == 0:
        return 0.0
    
    # Make a copy to avoid modifying the original set
    formulas = gradient_formulas.copy()
    total_grad = 0.0
    count = 0
    
    for gradient_polynomial in formulas:
        try:
            grad_val = gradient_polynomial.evaluate(values)
            total_grad += float(grad_val)
            count += 1
        except:
            pass
    
    return total_grad / count if count > 0 else 0.0

# Helper function to clip parameter values to valid probability range
def clip_probability(value, epsilon=1e-6):
    """Clip value to [epsilon, 1-epsilon] to maintain valid probabilities."""
    return max(epsilon, min(1.0 - epsilon, value))

### Momentum-based Gradient Descent

**Theoretical Foundation:**

Momentum introduces a "velocity" term that accumulates gradients over time, inspired by physics where a ball rolling down a hill accumulates momentum.

**Update Rules:**
```
v_t = β v_{t-1} + η ∇f(θ_t)
θ_{t+1} = θ_t - v_t
```

Where:
- $v_t$: velocity (exponentially weighted moving average of gradients)
- $\beta$: momentum coefficient (typically 0.9, meaning 90% of previous velocity is retained)

**Mathematical Intuition:**

The velocity at time $t$ is:
```
v_t = η(∇f(θ_t) + β∇f(θ_{t-1}) + β²∇f(θ_{t-2}) + β³∇f(θ_{t-3}) + ...)
```

This creates an exponentially decaying sum of past gradients.

**Key Properties:**

1. **Acceleration in consistent directions**: If gradients point in the same direction across iterations, velocity builds up
2. **Dampening oscillations**: Opposing gradients cancel out, reducing oscillation
3. **Escaping local minima**: Accumulated momentum can carry optimization through small local minima
4. **Convergence rate (speedup)**: Can achieve $O(1/t^2)$ convergence vs $O(1/t)$ for standard GD in some settings

**Why it works:**
- In ravines, gradients oscillate perpendicular to the optimum direction but point consistently toward optimum
- Momentum accumulates along the ravine while oscillations cancel out
- Effectively increases step size in low-curvature directions and decreases in high-curvature directions

---

Classic gradient descent with momentum accelerates convergence by accumulating past gradients:

In [ ]:
def gradient_descent_momentum(parametric_model, formula, parameters,
                             learning_rate=0.01, momentum=0.9, 
                             max_iterations=100, convergence_threshold=1e-6,
                             minimize=True):
    """
    Gradient Descent with Momentum
    
    Args:
        parametric_model: Storm parametric model
        formula: Property formula to optimize
        parameters: Model parameters to optimize
        learning_rate: Step size for updates
        momentum: Momentum coefficient (typically 0.9)
        max_iterations: Maximum optimization iterations
        convergence_threshold: Stop if objective change < threshold
        minimize: True to minimize, False to maximize objective
    """
    # Initialize parameters
    current_values = {p: stormpy.RationalRF(0.5) for p in parameters}
    instantiator = stormpy.pars.ModelInstantiator(parametric_model)
    
    # Initialize momentum terms (velocity)
    velocity = {p: 0.0 for p in parameters}
    
    prev_objective = None
    
    for iteration in range(max_iterations):
        # Instantiate model with current parameters
        concrete_model = instantiator.instantiate(current_values)
        
        # Model check to get objective value
        result = stormpy.model_checking(concrete_model, formula)
        objective = float(result.at(concrete_model.initial_states[0]))
        
        # Check convergence
        if prev_objective is not None and abs(objective - prev_objective) < convergence_threshold:
            print(f"Converged at iteration {iteration}")
            break
        
        # Compute gradients
        gradients = compute_gradients(parametric_model, parameters)
        
        # Update parameters using momentum
        for param in parameters:
            grad_value = evaluate_gradient_improved(gradients[param], current_values)
            
            # Apply gradient direction based on minimize/maximize
            grad_direction = grad_value if minimize else -grad_value
            
            # Update velocity with momentum
            velocity[param] = momentum * velocity[param] + learning_rate * grad_direction
            
            # Update parameter value
            new_value = float(current_values[param]) - velocity[param]
            current_values[param] = stormpy.RationalRF(clip_probability(new_value))
        
        if iteration % 10 == 0:
            print(f"Iteration {iteration}: Objective = {objective:.6f}")
        
        prev_objective = objective
    
    print(f"Final Objective: {objective:.6f}")
    return current_values

### RMSprop Optimizer (Root Mean Square Propagation)

**Theoretical Foundation:**

RMSprop adapts the learning rate for each parameter based on the magnitude of recent gradients. Developed by Geoffrey Hinton to address the problem of radically different gradient scales across parameters.

**Update Rules:**
```
E[g²]_t = ρ E[g²]_{t-1} + (1-ρ) (∇f(θ_t))²
θ_{t+1} = θ_t - η / √(E[g²]_t + ε) · ∇f(θ_t)
```

Where:
- $E[g^2]_t$: exponentially weighted moving average of squared gradients
- $\rho$: decay rate (typically 0.9)
- $\epsilon$: small constant for numerical stability (typically $10^{-8}$)

**Mathematical Intuition:**

For each parameter $\theta_i$:
- If gradients have been consistently large → $E[g^2]_i$ is large → effective learning rate $\eta/\sqrt{E[g^2]_i}$ is small
- If gradients have been consistently small → $E[g^2]_i$ is small → effective learning rate is large

**Key Properties:**

1. **Adaptive learning rates**: Each parameter gets its own effective learning rate
2. **Normalization**: Dividing by RMS of gradients normalizes the update step
3. **Handles different scales**: Parameters with different gradient magnitudes converge at similar rates
4. **Non-stationary objectives**: Works well when objective function changes over time

**Connection to Second-Order Methods:**

RMSprop approximates diagonal preconditioning. The ideal Newton's method update is:
```
θ_{t+1} = θ_t - H⁻¹ ∇f(θ_t)
```

Where H is the Hessian (second derivative matrix). RMSprop approximates the diagonal of $H^{-1}$ using gradient statistics.

**Why it works:**
- Normalizes gradient components by their typical magnitude
- Prevents parameters with large gradients from dominating updates
- Effectively rescales the parameter space to be more spherical

---

RMSprop uses adaptive learning rates based on a moving average of squared gradients:

In [ ]:
def rmsprop_optimization(parametric_model, formula, parameters,
                        learning_rate=0.001, decay_rate=0.9, epsilon=1e-8,
                        max_iterations=100, convergence_threshold=1e-6,
                        minimize=True):
    """
    RMSprop Optimizer with adaptive learning rates
    
    Args:
        parametric_model: Storm parametric model
        formula: Property formula to optimize
        parameters: Model parameters to optimize
        learning_rate: Initial learning rate
        decay_rate: Decay rate for squared gradient moving average (typically 0.9)
        epsilon: Small constant for numerical stability
        max_iterations: Maximum optimization iterations
        convergence_threshold: Stop if objective change < threshold
        minimize: True to minimize, False to maximize objective
    """
    # Initialize parameters
    current_values = {p: stormpy.RationalRF(0.5) for p in parameters}
    instantiator = stormpy.pars.ModelInstantiator(parametric_model)
    
    # Initialize squared gradient accumulator
    squared_grad = {p: 0.0 for p in parameters}
    
    prev_objective = None
    
    for iteration in range(max_iterations):
        # Instantiate model with current parameters
        concrete_model = instantiator.instantiate(current_values)
        
        # Model check to get objective value
        result = stormpy.model_checking(concrete_model, formula)
        objective = float(result.at(concrete_model.initial_states[0]))
        
        # Check convergence
        if prev_objective is not None and abs(objective - prev_objective) < convergence_threshold:
            print(f"Converged at iteration {iteration}")
            break
        
        # Compute gradients
        gradients = compute_gradients(parametric_model, parameters)
        
        # Update parameters using RMSprop
        for param in parameters:
            grad_value = evaluate_gradient_improved(gradients[param], current_values)
            
            # Apply gradient direction based on minimize/maximize
            grad_direction = grad_value if minimize else -grad_value
            
            # Update squared gradient moving average
            squared_grad[param] = decay_rate * squared_grad[param] + (1 - decay_rate) * grad_direction**2
            
            # Compute adaptive learning rate
            adaptive_lr = learning_rate / (np.sqrt(squared_grad[param]) + epsilon)
            
            # Update parameter value
            new_value = float(current_values[param]) - adaptive_lr * grad_direction
            current_values[param] = stormpy.RationalRF(clip_probability(new_value))
        
        if iteration % 10 == 0:
            print(f"Iteration {iteration}: Objective = {objective:.6f}")
        
        prev_objective = objective
    
    print(f"Final Objective: {objective:.6f}")
    return current_values

### Adam Optimizer (Adaptive Moment Estimation)

**Theoretical Foundation:**

Adam combines the benefits of momentum (first moment) and RMSprop (second moment) with bias correction. Published by Kingma & Ba (2014), it's one of the most widely used optimizers in deep learning.

**Update Rules:**
```
m_t = β₁ m_{t-1} + (1-β₁) ∇f(θ_t)           [First moment estimate]
v_t = β₂ v_{t-1} + (1-β₂) (∇f(θ_t))²        [Second moment estimate]

m̂_t = m_t / (1 - β₁ᵗ)                        [Bias correction for first moment]
v̂_t = v_t / (1 - β₂ᵗ)                        [Bias correction for second moment]

θ_{t+1} = θ_t - η m̂_t / (√v̂_t + ε)
```

Where:
- $\beta_1$: exponential decay rate for first moment (typically 0.9)
- $\beta_2$: exponential decay rate for second moment (typically 0.999)
- $m_t$: biased first moment estimate (mean of gradients)
- $v_t$: biased second moment estimate (uncentered variance of gradients)
- $\hat{m}_t$, $\hat{v}_t$: bias-corrected estimates

**Mathematical Intuition:**

**First Moment ($m_t$):** Exponentially weighted moving average of gradients
```
m_t = (1-β₁) Σᵢ₌₁ᵗ β₁^(t-i) ∇f(θᵢ)
```

**Second Moment ($v_t$):** Exponentially weighted moving average of squared gradients
```
v_t = (1-β₂) Σᵢ₌₁ᵗ β₂^(t-i) (∇f(θᵢ))²
```

**Why Bias Correction?**

When initialized at $m_0 = 0$, $v_0 = 0$:
- Early estimates are biased toward zero
- $E[m_t] \neq E[\nabla f(\theta_t)]$ in early iterations
- Bias correction factor $(1-\beta^t)$ compensates for this initialization bias

**Key Properties:**

1. **Combines momentum + adaptive learning**: Gets benefits of both methods
2. **Bias correction**: Ensures accurate estimates especially early in training
3. **Invariance to gradient rescaling**: Update magnitude is invariant to gradient scale
4. **Bounded step sizes**: Step size has bounds related to learning rate

**Convergence Guarantees:**

For convex functions, Adam achieves regret bound:
```
R(T) ≤ O(√T)
```

This means average regret goes to zero at rate $O(1/\sqrt{T})$.

**Why it works:**
- First moment (momentum) provides direction information and noise reduction
- Second moment (adaptive learning) provides scaling information
- Bias correction prevents overly large steps early on
- Combination is robust to hyperparameter choices

**Comparison of effective step sizes:**

| Optimizer | Effective Step Size |
|-----------|-------------------|
| SGD | $\eta$ |
| Momentum | $\eta$ (amplified by velocity) |
| RMSprop | $\eta / \sqrt{E[g^2]}$ |
| Adam | $\eta \cdot \hat{m} / \sqrt{\hat{v}}$ |

---

Adam (Adaptive Moment Estimation) combines momentum and adaptive learning rates for robust optimization:

In [ ]:
def adam_optimization(parametric_model, formula, parameters,
                     learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8,
                     max_iterations=100, convergence_threshold=1e-6,
                     minimize=True, gradient_clip=None):
    """
    Adam Optimizer - combines momentum and adaptive learning rates
    
    Args:
        parametric_model: Storm parametric model
        formula: Property formula to optimize
        parameters: Model parameters to optimize
        learning_rate: Learning rate (typically 0.001)
        beta1: Exponential decay rate for first moment (typically 0.9)
        beta2: Exponential decay rate for second moment (typically 0.999)
        epsilon: Small constant for numerical stability
        max_iterations: Maximum optimization iterations
        convergence_threshold: Stop if objective change < threshold
        minimize: True to minimize, False to maximize objective
        gradient_clip: Maximum gradient magnitude (None for no clipping)
    """
    # Initialize parameters
    current_values = {p: stormpy.RationalRF(0.5) for p in parameters}
    instantiator = stormpy.pars.ModelInstantiator(parametric_model)
    
    # Initialize first moment (momentum) and second moment (adaptive lr) estimates
    m = {p: 0.0 for p in parameters}  # First moment
    v = {p: 0.0 for p in parameters}  # Second moment
    
    prev_objective = None
    
    for iteration in range(max_iterations):
        # Instantiate model with current parameters
        concrete_model = instantiator.instantiate(current_values)
        
        # Model check to get objective value
        result = stormpy.model_checking(concrete_model, formula)
        objective = float(result.at(concrete_model.initial_states[0]))
        
        # Check convergence
        if prev_objective is not None and abs(objective - prev_objective) < convergence_threshold:
            print(f"Converged at iteration {iteration}")
            break
        
        # Compute gradients
        gradients = compute_gradients(parametric_model, parameters)
        
        # Update parameters using Adam
        for param in parameters:
            grad_value = evaluate_gradient_improved(gradients[param], current_values)
            
            # Apply gradient direction based on minimize/maximize
            grad_direction = grad_value if minimize else -grad_value
            
            # Gradient clipping (optional)
            if gradient_clip is not None:
                grad_direction = np.clip(grad_direction, -gradient_clip, gradient_clip)
            
            # Update biased first moment estimate
            m[param] = beta1 * m[param] + (1 - beta1) * grad_direction
            
            # Update biased second raw moment estimate
            v[param] = beta2 * v[param] + (1 - beta2) * grad_direction**2
            
            # Compute bias-corrected first moment estimate
            m_hat = m[param] / (1 - beta1**(iteration + 1))
            
            # Compute bias-corrected second raw moment estimate
            v_hat = v[param] / (1 - beta2**(iteration + 1))
            
            # Update parameter value
            new_value = float(current_values[param]) - learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
            current_values[param] = stormpy.RationalRF(clip_probability(new_value))
        
        if iteration % 10 == 0:
            print(f"Iteration {iteration}: Objective = {objective:.6f}")
        
        prev_objective = objective
    
    print(f"Final Objective: {objective:.6f}")
    return current_values

### Usage Examples

Test the different optimizers on your parametric model:

In [ ]:
# Test 1: Gradient Descent with Momentum
print("=" * 60)
print("Testing Gradient Descent with Momentum")
print("=" * 60)

optimal_params_momentum = gradient_descent_momentum(
    parametric_model, 
    exec.property[0].raw_formula, 
    parameters,
    learning_rate=0.01,
    momentum=0.9,
    max_iterations=100,
    minimize=True  # Set to False if maximizing the objective
)

# Evaluate final result
instantiator = stormpy.pars.ModelInstantiator(parametric_model)
final_model_momentum = instantiator.instantiate(optimal_params_momentum)
result_momentum = stormpy.model_checking(final_model_momentum, exec.property[0].raw_formula)
print(f"\nFinal result with Momentum: {result_momentum.at(final_model_momentum.initial_states[0])}")

In [ ]:
# Test 2: RMSprop Optimizer
print("\n" + "=" * 60)
print("Testing RMSprop Optimizer")
print("=" * 60)

optimal_params_rmsprop = rmsprop_optimization(
    parametric_model, 
    exec.property[0].raw_formula, 
    parameters,
    learning_rate=0.01,
    decay_rate=0.9,
    max_iterations=100,
    minimize=True
)

# Evaluate final result
final_model_rmsprop = instantiator.instantiate(optimal_params_rmsprop)
result_rmsprop = stormpy.model_checking(final_model_rmsprop, exec.property[0].raw_formula)
print(f"\nFinal result with RMSprop: {result_rmsprop.at(final_model_rmsprop.initial_states[0])}")

In [ ]:
# Test 3: Adam Optimizer (Recommended)
print("\n" + "=" * 60)
print("Testing Adam Optimizer")
print("=" * 60)

optimal_params_adam = adam_optimization(
    parametric_model, 
    exec.property[0].raw_formula, 
    parameters,
    learning_rate=0.01,
    beta1=0.9,
    beta2=0.999,
    max_iterations=100,
    minimize=True,
    gradient_clip=10.0  # Clip gradients to prevent instability
)

# Evaluate final result
final_model_adam = instantiator.instantiate(optimal_params_adam)
result_adam = stormpy.model_checking(final_model_adam, exec.property[0].raw_formula)
print(f"\nFinal result with Adam: {result_adam.at(final_model_adam.initial_states[0])}")

### Comparison and Analysis

Compare the performance of different optimizers:

In [ ]:
# Compare optimizer results
print("\n" + "=" * 60)
print("OPTIMIZER COMPARISON")
print("=" * 60)

results_comparison = {
    "Momentum": float(result_momentum.at(final_model_momentum.initial_states[0])),
    "RMSprop": float(result_rmsprop.at(final_model_rmsprop.initial_states[0])),
    "Adam": float(result_adam.at(final_model_adam.initial_states[0]))
}

for optimizer_name, objective_value in results_comparison.items():
    print(f"{optimizer_name:15s}: {objective_value:.6f}")

best_optimizer = min(results_comparison, key=results_comparison.get)
print(f"\nBest optimizer: {best_optimizer} with objective = {results_comparison[best_optimizer]:.6f}")

### Additional Features:
- **Gradient clipping**: Prevents exploding gradients
- **Convergence detection**: Stops when objective change is below threshold
- **Probability bounds**: Ensures parameters stay in valid $[0,1]$ range
- **Improved gradient evaluation**: Properly handles multiple derivatives per parameter

### Conclusion
At each step the derivatives extracted from stormpy are zero for most parameters, hence they seem to not affect the reward objective, which is in fact unrealistic. The problem might be too complex for symbolic derivatives and potentially not be captured by gradient-based methods. Alternatives: random search or evolutionary algorithms.

### Comparative Analysis

**Convergence Behavior:**

```
Standard GD:    ━━━━━━━━━━→ (straight path, may oscillate)
Momentum:       ━━━━━╱━━→   (builds speed, overshoots less)
RMSprop:        ━┓━┓━→      (adaptive steps)
Adam:           ━━━━━━→     (smooth, fast, adaptive)
```

**When to Use Each:**

- **Standard GD**: Convex problems with well-conditioned Hessian, theoretical analysis
- **Momentum**: Problems with ravines, want faster convergence, can tolerate some overshoot
- **RMSprop**: Non-stationary objectives, different parameter scales, recurrent neural networks
- **Adam**: Default choice for most problems, especially when gradient is sparse or noisy

**Hyperparameter Sensitivity:**

- **Standard GD**: Very sensitive to learning rate $\eta$
- **Momentum**: Moderately sensitive to $\eta$ and $\beta$
- **RMSprop**: Less sensitive, but $\rho$ affects adaptation speed
- **Adam**: Least sensitive, default values ($\beta_1=0.9$, $\beta_2=0.999$) work well across many problems

---

### Memory-less POMDP Controller Problem

**Why these matter for parametric Markov chains:**

1. **Different parameter influence**: Some FSC parameters may have much larger impact on objective than others → RMSprop/Adam handle this automatically

2. **Sparse gradients**: Most parameters show zero derivatives in the output logs → Adam's momentum can help explore despite sparse gradients

3. **Parameter correlations**: POMDP controller parameters may have complex interdependencies → Momentum helps navigate correlated parameter space

4. **Probability constraints**: Parameters must stay in $[0,1]$ → Adaptive learning rates prevent overshooting bounds

**Theoretical limitation in this case:**

If `stormpy.pars.gather_derivatives` returns mostly empty sets (zero symbolic derivatives), it suggests:
- Parameters may not appear in reachability probabilities (derivative truly is zero)
- Symbolic differentiation may not capture all dependencies
- Need to verify that FSC parameters actually influence the verification property


### 10. Practical Hyperparameter Guidance

**Default Starting Points:**

| Method | Learning Rate | Other Parameters | When to Use |
|--------|---------------|------------------|-------------|
| SGD | 0.1 - 0.01 | - | Simple convex problems |
| Momentum | 0.01 - 0.001 | β = 0.9 | Ill-conditioned problems |
| RMSprop | 0.001 - 0.0001 | ρ = 0.9 | RNNs, non-stationary |
| Adam | 0.001 - 0.0001 | β₁=0.9, β₂=0.999 | Default choice |

**Hyperparameter Tuning Strategy:**

1. Start with defaults (especially for Adam)
2. If diverging: decrease learning rate by 10× 
3. If too slow: increase learning rate by 3×
4. For momentum methods: increase β for smoother updates
5. For Adam: rarely need to change β₁, β₂

**Common Failure Modes:**

- **Oscillation**: Learning rate too high → decrease η
- **Slow progress**: Learning rate too low or trapped in plateau → increase η or add noise
- **Early stopping**: Momentum overshoots → decrease β or η
- **Exploding values**: Gradient explosion → add gradient clipping or decrease η

### 11. Key Research Papers and References

**Foundational Papers:**

1. **Momentum:**
   - Polyak, B. T. (1964). "Some methods of speeding up the convergence of iteration methods"
   - Nesterov, Y. (1983). "A method for solving the convex programming problem with convergence rate O(1/k²)"

2. **RMSprop:**
   - Hinton, G., Srivastava, N., & Swersky, K. (2012). "Neural Networks for Machine Learning" - Lecture 6a
   - Tieleman, T., & Hinton, G. (2012). "Lecture 6.5-rmsprop: Divide the gradient by a running average of its recent magnitude"

3. **Adam:**
   - Kingma, D. P., & Ba, J. (2014). "Adam: A Method for Stochastic Optimization" - ICLR 2015
   - [arXiv:1412.6980](https://arxiv.org/abs/1412.6980)

**Survey Papers:**

- Ruder, S. (2016). "An overview of gradient descent optimization algorithms" - [arXiv:1609.04747](https://arxiv.org/abs/1609.04747)
- Bottou, L., Curtis, F. E., & Nocedal, J. (2018). "Optimization Methods for Large-Scale Machine Learning" - SIAM Review

**For POMDP Controllers:**

- Junges, S., et al. (2018). "Finite-State Controllers of POMDPs using Parameter Synthesis" - UAI 2018
- Carr, S., et al. (2019). "Counterexample-Guided Strategy Improvement for POMDPs Using Recurrent Neural Networks" - IJCAI 2019

---

### Gradient-Based Methods May Not Be Sufficient

**Current Challenge:**

From the notebook output, most parameters have `set()` (empty) derivatives, with only `p1_0` and `p1_1` showing non-zero symbolic derivatives. This indicates:

**Why gradient descent may struggle:**

1. **Structural zeros**: Parameters that don't appear in transition probabilities have genuinely zero derivatives
2. **Discrete structure**: POMDP finite-state controllers may have discontinuous objective landscapes
3. **Symbolic limitations**: `stormpy.pars.gather_derivatives` may only compute derivatives for parameters that appear explicitly in transition functions

If sensitivity is near zero for all parameters, the FSC parameterization may not be suitable for this POMDP.

### Diagnostic: Test Parameter Sensitivity

Before investing in optimization, verify that parameters actually influence the objective:

In [10]:
# Test parameter sensitivity - does changing parameters affect the objective?
import random

print("Parameter Sensitivity Analysis")
print("=" * 80)

# Test a random sample of parameters
params_to_test = random.sample(list(parameters), min(10, len(parameters)))
test_values = [0.1, 0.3, 0.5, 0.7, 0.9]

sensitivity_results = {}
instantiator = stormpy.pars.ModelInstantiator(parametric_model)

for param_to_test in params_to_test:
    objectives = []
    
    for val in test_values:
        test_params = {p: stormpy.RationalRF(0.5) for p in parameters}
        test_params[param_to_test] = stormpy.RationalRF(val)
        test_model = instantiator.instantiate(test_params)
        result = stormpy.model_checking(test_model, exec.property[0].raw_formula)
        objectives.append(float(result.at(test_model.initial_states[0])))
    
    sensitivity = max(objectives) - min(objectives)
    sensitivity_results[str(param_to_test)] = sensitivity
    
    print(f"\nParameter: {param_to_test}")
    print(f"  Values tested: {test_values}")
    print(f"  Objectives:    {[f'{obj:.4f}' for obj in objectives]}")
    print(f"  Sensitivity:   {sensitivity:.6f} {'✓ SENSITIVE' if sensitivity > 1e-6 else '✗ NOT SENSITIVE'}")

print("\n" + "=" * 80)
print("Summary:")
sensitive_params = sum(1 for s in sensitivity_results.values() if s > 1e-6)
print(f"Sensitive parameters: {sensitive_params}/{len(params_to_test)}")
print(f"Max sensitivity: {max(sensitivity_results.values()):.6f}")
print(f"Mean sensitivity: {sum(sensitivity_results.values())/len(sensitivity_results):.6f}")

Parameter Sensitivity Analysis

Parameter: p14_0
  Values tested: [0.1, 0.3, 0.5, 0.7, 0.9]
  Objectives:    ['623.4444', '625.9048', '630.3333', '640.6667', '692.3333']
  Sensitivity:   68.888889 ✓ SENSITIVE

Parameter: p23_0
  Values tested: [0.1, 0.3, 0.5, 0.7, 0.9]
  Objectives:    ['469.4889', '526.9333', '630.3333', '871.6000', '2077.9333']
  Sensitivity:   1608.444444 ✓ SENSITIVE

Parameter: p16_0
  Values tested: [0.1, 0.3, 0.5, 0.7, 0.9]
  Objectives:    ['580.5111', '598.3048', '630.3333', '705.0667', '1078.7333']
  Sensitivity:   498.222222 ✓ SENSITIVE

Parameter: p10_0
  Values tested: [0.1, 0.3, 0.5, 0.7, 0.9]
  Objectives:    ['471.8889', '528.4762', '630.3333', '868.0000', '2056.3333']
  Sensitivity:   1584.444444 ✓ SENSITIVE

Parameter: p8_0
  Values tested: [0.1, 0.3, 0.5, 0.7, 0.9]
  Objectives:    ['494.3778', '542.9333', '630.3333', '834.2667', '1853.9333']
  Sensitivity:   1359.555556 ✓ SENSITIVE

Parameter: p6_0
  Values tested: [0.1, 0.3, 0.5, 0.7, 0.9]
  Objecti